In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import seaborn as sns
import os
import warnings
warnings.filterwarnings("ignore")


for dirname, _, filenames in os.walk('/kaggle/input'): # new way of reading comppetition input  
    for filename in filenames:                         # csv files
        file_name = os.path.join(dirname, filename)    # if you liked, then upvote
        print(file_name)
        if "Finland" in file_name:                   # to make notebook popular among Kagglers
            fi_df = pd.read_csv(file_name)                # 'df' represents for train dataset
        elif "Norway" in file_name:
            no_df = pd.read_csv(file_name)              # 'test' represents for test dataset
        elif "Sweden" in file_name:
            se_df = pd.read_csv(file_name)            # 'sample' represents for sample submission dataset
        elif "train.csv" in file_name:                   # to make notebook popular among Kagglers
            df = pd.read_csv(file_name)                # 'df' represents for train dataset
        elif "test.csv" in file_name:
            test = pd.read_csv(file_name)              # 'test' represents for test dataset
        elif "submission.csv" in file_name:
            sample = pd.read_csv(file_name) 

In [ ]:
fi_df.Date = pd.to_datetime(fi_df.Date)
se_df.Date = pd.to_datetime(se_df.Date)
no_df.Date = pd.to_datetime(no_df.Date)

In [ ]:
fi_df.set_index("Date", inplace=True)
se_df.set_index("Date", inplace=True)
no_df.set_index("Date", inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])
test['date'] = pd.to_datetime(test['date'])
def clean_store(item):
    return item[6:7] # 'KaggleMart' => 'M'
def clean_product(product):
    return product[7:8] # 'Kaggle Hat' => 'H'
def clean_country(items):
    return items[0] # 'Finland' => 'F'
df.store = df.store.apply(clean_store)
test.store = test.store.apply(clean_store)
df['product'] = df['product'].apply(clean_product)
test['product'] = test['product'].apply(clean_product)
df['country'] = df['country'].apply(clean_country)
test['country'] = test['country'].apply(clean_country)
df['hybrid'] =  df['country'] + df['store'] +df['product']
test['hybrid'] =  test['country'] + test['store'] + test['product']
best = df.drop(["country", "store", "product", 'row_id'], axis=1)
X_valid = test.drop(["country", "store", "product", 'row_id'], axis=1)
best = df.set_index(['date', 'hybrid']).unstack()['num_sold']
#X_test= X_test.set_index(['date', 'hybrid']).unstack()['hybrid']



In [ ]:
best_df = best
best_df['mymean'] = 0
for i in best_df.columns:
    best_df['mymean']+=best_df[i]
best_df['mymean'] = best_df['mymean']/len(best_df.columns)

In [ ]:
inter2 = fi_df.index.union( no_df.index)

In [ ]:
inter2 = inter2.union(no_df.index)

In [ ]:
holidays = best_df.loc[best_df.index.intersection(inter2)]

In [ ]:
holidays['mymean'] = 0
for i in holidays.columns:
    holidays['mymean']+=holidays[i]
holidays['mymean'] = holidays['mymean']/len(holidays.columns)

In [ ]:
best_df.mymean.index

In [ ]:

plt.figure(figsize=(36,12))
sns.lineplot(x=best_df.loc[best_df.index.year>2014].mymean.index, 
                           y=best_df.loc[best_df.index.year>2014].mymean, label="sales rate")
plt.scatter(x=best_df.loc[(best_df.index.year>2014) & (best_df.index.day_of_week == 5)].mymean.index, 
            y=best_df.loc[(best_df.index.year>2014) & (best_df.index.day_of_week == 5)].mymean, c="yellow", label="Saturday")
plt.scatter(x=best_df.loc[(best_df.index.year>2014) & (best_df.index.day_of_week == 6)].mymean.index, 
            y=best_df.loc[(best_df.index.year>2014) & (best_df.index.day_of_week == 6)].mymean, c="pink", label="Sunday")
plt.scatter(x=holidays.loc[(holidays.index.year>2014)].index, 
                            y=holidays.loc[(holidays.index.year>2014)].mymean, c="red", label="Holiday")

plt.legend()